In [38]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [39]:
!cat /usr/local/cuda/version.txt
!pip install pydash
!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130


In [3]:
!rm -rf outputs

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("downloads/47k_train_no_acl.csv")
len(df)

41561

In [5]:
blacklist=["abstract", "discussion", "background", "summary", "rationale", "introduction", "purpose", "objective" ]
blacklist += [t + ":" for t in blacklist]
def remove_special_words(text, blacklist=blacklist ):
    words = text.split(" ")
    try:
        first=words[0]
    except:
        return text
    
    if any(first.strip().lower() in string for string in blacklist):
        return " ".join(words[1:]).strip()
    return text

remove_special_words("ABSTRACT   The northern corn rootworm")

'The northern corn rootworm'

In [6]:
df["text"] = df["text"].map(remove_special_words)
df["source"].unique()

array(['predicts_acl_pm2500_vocabed', 'Oct1_clinical_studies_pm',
       'oct3_labels', 'labels_oct7',
       ' limitation|problem ... with ... l=be that', 'a ... drawback of',
       'a {negative_adjectives} ...  {problem_nouns}', 'pm_OBJECTIVE',
       'pm_METHODS', 'pm_RESULTS', 'pm_CONCLUSIONS', 'biology1k_050522',
       'psych1k_100522'], dtype=object)

In [7]:
#downsample medical data in training set
df = df[df["source"].isin(["oct3_labels", "labels_oct7", "psych1k_100522", "biology1k_050522", "predicts_acl_pm2500_vocabed"])]

In [8]:
df= df[["text", "labels"]]

In [9]:
df =df.dropna()
df = df.rename(columns={"labels": "label"})

In [10]:
df= df[["text", "label"]]
df.label = df.label.map(int)

In [11]:
df

,text,label
0,"While often idiopathic, anterior ischemic opti...",0
1,Primary cutaneous B-cell lymphomas (PCBCLs) ar...,0
2,The insufflation of gas during colonoscopy lea...,0
3,The aim of the present study is to investigate...,0
4,The aim of this study was to assess predictive...,0
...,...,...
41556,Mobbing is actually a social problem.,0
41557,Students starting off on a tertiary education ...,1
41558,Many individuals with vision impairment experi...,1
41559,Considering China is facing a precipitous decl...,0


In [12]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [13]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)
# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1950 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 1958 features created from 1950 samples.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/245 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/487 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 487 features created from 487 samples.


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.6929873771078452, 'tp': 269, 'tn': 145, 'fp': 57, 'fn': 16, 'acc': 0.8501026694045175, 'eval_loss': 0.349631943529652}


In [14]:
result

{'mcc': 0.6929873771078452,
 'tp': 269,
 'tn': 145,
 'fp': 57,
 'fn': 16,
 'acc': 0.8501026694045175,
 'eval_loss': 0.349631943529652}

In [41]:
import pandas as pd

In [42]:
dfx = pd.read_csv("downloads/s2c_5M_bioadjacent.csv",  lineterminator='\n', error_bad_lines=False)


In [43]:
dfx = dfx.dropna(subset=["text"])
dfx=dfx[dfx["text"].str.len() > 15]

dfx.head()

dfx = dfx[dfx.fields.isin(["Medicine", 'Environmental Science', 'Sociology', 'Psychology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

dfx["text"] = dfx["text"].map(remove_special_words)

len(dfx)

3163352

In [44]:
dfx["text"] = dfx["text"].astype(str)

In [45]:
dfx = dfx.sample(frac=0.2)

In [46]:
len(dfx)

632670

In [47]:
dfx.to_csv("downloads/sample_bioadjacent600k.csv", index = False, header=True)

In [48]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
        with open('predicts_backup.txt', 'w') as f:
            for item in l:
                f.write("%s\n" % item)
    return l

In [ ]:
predicts = stepwise(20000, dfx["text"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 20000


  0%|          | 0/20000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 20000 features created from 20000 samples.


  0%|          | 0/2532 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


500000 520000


  0%|          | 0/20000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1294 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1141 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 20000 features created from 20000 samples.


  0%|          | 0/2534 [00:00<?, ?it/s]

In [56]:
dfx

,index,text,s2Url,doi,journalName,title,fieldsOfStudy,fields
3511373,1410258,Vibration events collected from piezo-electric...,https://semanticscholar.org/paper/f2d437dea6f7...,NaN,NaN,Transmission of Polistes Vibrational Signals A...,['Biology'],Biology
1771550,3807306,Many of the ovules excised 4∼5 days after anth...,https://semanticscholar.org/paper/dd3f584b19c7...,10.1270/JSBBS1951.19.373,Breeding Science,IN VITRO CULTURE OF EXCISED OVULES OF WHITE CL...,['Biology'],Biology
2270346,4878973,"Since their discovery in 1991, carbon nanotube...",https://semanticscholar.org/paper/a1b1bd6960c0...,10.1109/TIA.2004.834051,IEEE Transactions on Industry Applications,Electronic properties of single-wall carbon na...,['Materials Science'],Materials Science
3551758,1497454,Arthroscopic shoulder stabilization is a widel...,https://semanticscholar.org/paper/45734cfacf75...,NaN,NaN,Open or arthroscopic shoulder stabilization – ...,['Medicine'],Medicine
4496937,3526698,We introduce an effective iterative Least-Squa...,https://semanticscholar.org/paper/f40009972c0a...,10.1071/ASEG2018abW8_1A,ASEG Extended Abstracts,Broadband Least-Squares Wave-Equation Migration,['Geology'],Geology
...,...,...,...,...,...,...,...,...
1432850,3079957,Bambu alas adalah salah satu jenis bambu endem...,https://semanticscholar.org/paper/dd752da4ae4c...,10.14203/BKR.V17I1.134,Buletin Kebun Raya,"Studi Populasi Dinochloa Sepang, Bambu Endemik...",['Biology'],Biology
561008,1205527,Understanding of the RNA editing process has b...,https://semanticscholar.org/paper/17b05580f452...,10.1371/journal.pgen.1004274,PLoS Genetics,RNA Editome in Rhesus Macaque Shaped by Purify...,"['Biology', 'Medicine']",Biology
4730984,4029406,Photodynamic therapy involves the administrati...,https://semanticscholar.org/paper/92ef755a0fbd...,10.1590/S0365-05962009000500002,Anais Brasileiros De Dermatologia,Terapia fotodinâmica em dermatologia: princípi...,['Medicine'],Medicine
3194686,728820,QUALITY NURSING home care has been and continu...,https://semanticscholar.org/paper/6815c50456da...,10.1097/01.NCQ.0000303799.84585.a1,Journal of Nursing Care Quality,The Organizational Quality Frontier and Essent...,['Medicine'],Medicine


In [59]:
with open('predicts_backup.txt', 'r') as f:
    preds = f.read().split()#f.write("%s\n" % item)
                

In [60]:
len(preds)

632670

In [ ]:
#check if not already in dataset

In [61]:
dfo = pd.read_csv("downloads/47k_train_no_acl.csv")


In [62]:
len(dfx)

632670

In [63]:
#dfx = dfx[:3000000]
dfx["predicts"] = preds

In [64]:
dfx = dfx[dfx["predicts"]==1]
dfx = dfx[dfx["text"] != "nan"]
dfx = dfx[dfx["text"].str.len() > 14]
len(dfx)

0

In [65]:
dd = dfx[~dfx["doi"].isin(list(dfo["doi"]))]
dfx=dd

In [ ]:
#check that not in final110k_1605

In [ ]:
dfx.to_csv("downloads/pos_preds_12062022.csv", index = False, header=True)

In [ ]:
dfx[dfx["fieldsOfStudy"].isin(['Environmental Science', 'Sociology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [181]:
dt = pd.read_csv("core_pos.csv")
#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0
dc["authors"] = dc["authors"].map(eval)
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)
tt.to_csv("core_pos_min.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

